In [1]:
import pandas as pd
import os
import sys
from db_utils import RDSDatabaseConnector
from data_extraction import DataExtractor
from data_transformer import DataTransformer
from data_frame_info import DataFrameInfo

current_directory = os.getcwd()


We begin by first extracting our data from an RDS database 
Ensure you have a yaml_file initialised with the correct credentials to access the RDS Database


In [2]:
# Path to the credentials file
yaml_file = "credentials.yaml"

# Initialize the database connector
db_connector = RDSDatabaseConnector(yaml_file)


# Initialize the data extractor with the database connector's engine
data_extractor = DataExtractor(db_connector.engine)

# Task 3 Step 6: Extract data from the 'customer_activity' table
table_name = "customer_activity"
data = data_extractor.read_rds_table(table_name)


Our db_connector utilizes the credentials from the YAML file to initialize a SQLAlchemy engine for database connections. 
This engine is passed to our data_extractor, enabling it to retrieve the customer activity data. 

We save this data into a CSV file on our local machine to avoid repeatedly connecting to the database, thereby accelerating our exploratory data analysis process.

In [3]:
csv_file_path = os.path.join(current_directory, 'customer_activity.csv')
data_extractor.save_to_csv(data, csv_file_path) # Saving data to a CSV file

Next step will be to load the data onto our panda dataframe, and to gather information on the nature of our dataset
so we can decide what the best way to clean/transform it is.



In [4]:
# Loading our data onto a dataframe
df = pd.read_csv(csv_file_path)

# Display the first few rows of the dataframe
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   administrative            11760 non-null  float64
 1   administrative_duration   11418 non-null  float64
 2   informational             12330 non-null  int64  
 3   informational_duration    11994 non-null  float64
 4   product_related           11751 non-null  float64
 5   product_related_duration  12129 non-null  float64
 6   bounce_rates              12330 non-null  float64
 7   exit_rates                12330 non-null  float64
 8   page_values               12330 non-null  float64
 9   month                     12330 non-null  object 
 10  operating_systems         12319 non-null  object 
 11  browser                   12330 non-null  object 
 12  region                    12330 non-null  object 
 13  traffic_type              12330 non-null  object 
 14  visito

,administrative,administrative_duration,informational,informational_duration,product_related,product_related_duration,bounce_rates,exit_rates,page_values,month,operating_systems,browser,region,traffic_type,visitor_type,weekend,revenue
0,0.0,0.0,0,0.0,4.0,0.0,0.2000,0.200,0.0,May,Android,Google Chrome,North America,Twitter,Returning_Visitor,False,False
1,0.0,0.0,0,0.0,26.0,876.0,0.0000,0.026,0.0,Nov,Windows,Google Chrome,Western Europe,Google search,Returning_Visitor,True,False
2,2.0,99.4,0,0.0,19.0,368.0,0.0396,0.052,0.0,Sep,Windows,Google Chrome,Asia,Twitter,Returning_Visitor,False,False
3,0.0,0.0,0,0.0,20.0,1432.0,0.0248,0.028,1.8,Dec,Windows,Google Chrome,Western Europe,Instagram ads,Returning_Visitor,False,True
4,0.0,0.0,0,0.0,33.0,694.0,0.0141,0.032,0.0,May,Android,Google Chrome,North America,Instagram ads,Returning_Visitor,False,False


From our analysis we can see that we have various columns which dont have their data_types initilised, they are in "object" format. From the title of these columns they appear to be categorical, before converting it we will take a closer look.

In [15]:
df_info = DataFrameInfo(df)
print("Distinct values in each column:")
df[['month','operating_systems', 'browser', 'region', 'traffic_type', 'visitor_type']].nunique()

Distinct values in column 'month': 10


month                10
operating_systems     7
browser              13
region                9
traffic_type         19
visitor_type          3
dtype: int64

Our assumption is correct, they all have limited unique values, meaning it would make sense to change the column to categorical data type. This will not only conserve memory but allow us to do specialised analysis later on if we deem necessary. We will use our DataTransformer class, a class we created in 'data_transformer.py' to handle this dataset.

In [20]:
data_transformer = DataTransformer(df)
df = data_transformer.convert_columns(['month','operating_systems', 'browser', 'region', 'traffic_type', 'visitor_type'], dtype='category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   administrative            11760 non-null  float64 
 1   administrative_duration   11418 non-null  float64 
 2   informational             12330 non-null  int64   
 3   informational_duration    11994 non-null  float64 
 4   product_related           11751 non-null  float64 
 5   product_related_duration  12129 non-null  float64 
 6   bounce_rates              12330 non-null  float64 
 7   exit_rates                12330 non-null  float64 
 8   page_values               12330 non-null  float64 
 9   month                     12330 non-null  category
 10  operating_systems         12319 non-null  category
 11  browser                   12330 non-null  category
 12  region                    12330 non-null  category
 13  traffic_type              12330 non-null  cate

The next step will be to  take a closer look at our null values and decide how to handle it.
In the data_frame_info.py I have creaeted a clas with various methods to retrieve information. 

In [22]:
# Preforming further inspection
df_info = DataFrameInfo(df)
df_info.null_percentage()



administrative              4.622871
administrative_duration     7.396594
informational               0.000000
informational_duration      2.725061
product_related             4.695864
product_related_duration    1.630170
bounce_rates                0.000000
exit_rates                  0.000000
page_values                 0.000000
month                       0.000000
operating_systems           0.089213
browser                     0.000000
region                      0.000000
traffic_type                0.000000
visitor_type                0.000000
weekend                     0.000000
revenue                     0.000000
dtype: float64

From our analysis we can see there is a varying amount of null, with some columns having as little as 0.089% null to others as much as 7% we have a variety of options to deal with these values.

1. Drop them
2. Replace them with mean/mode/median
3. Apply more specialised analysis like KNN than use these values to impute them

Before preceding however we need to determine if our Null values are Missing At Random (MAR) or Not Missing At Random (NMAR). MAR datasets are ,..... an example is, ..., they have little to no effect on our  . And not missi



In [8]:
data_transformer = DataTransformer(df)
data_transformer.convert_columns(['month'],'int')

,administrative,administrative_duration,informational,informational_duration,product_related,product_related_duration,bounce_rates,exit_rates,page_values,month,operating_systems,browser,region,traffic_type,visitor_type,weekend,revenue
0,0.0,0.000000,0,0.00,4.0,0.0,0.2000,0.200,0.0,NaN,Android,Google Chrome,North America,Twitter,Returning_Visitor,False,False
1,0.0,0.000000,0,0.00,26.0,876.0,0.0000,0.026,0.0,NaN,Windows,Google Chrome,Western Europe,Google search,Returning_Visitor,True,False
2,2.0,99.400000,0,0.00,19.0,368.0,0.0396,0.052,0.0,NaN,Windows,Google Chrome,Asia,Twitter,Returning_Visitor,False,False
3,0.0,0.000000,0,0.00,20.0,1432.0,0.0248,0.028,1.8,NaN,Windows,Google Chrome,Western Europe,Instagram ads,Returning_Visitor,False,True
4,0.0,0.000000,0,0.00,33.0,694.0,0.0141,0.032,0.0,NaN,Android,Google Chrome,North America,Instagram ads,Returning_Visitor,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,NaN,134.000000,4,112.22,48.0,1898.0,0.0030,0.007,0.0,NaN,Windows,Google Chrome,Western Europe,Google search,Returning_Visitor,True,False
12326,5.0,NaN,1,0.00,67.0,2202.0,0.0276,0.069,0.0,NaN,Windows,Google Chrome,Southern Africa,Facebook ads,Returning_Visitor,False,False
12327,0.0,0.000000,0,0.00,13.0,316.0,0.0000,0.017,0.0,NaN,Windows,Google Chrome,South America,Google search,New_Visitor,False,False
12328,5.0,98.000000,0,0.00,46.0,1469.0,0.0442,0.069,0.0,NaN,Windows,Google Chrome,North America,Facebook ads,Returning_Visitor,False,False
